### IMPORTS

In [ ]:
import nltk
import spacy
from modules.utils import build_dataset, tune_logistic_regression, tune_svm, evaluate
import sklearn
from modules.preprocess import spacy_tokenizer, text_edit
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.neural_network import MLPClassifier

### LOAD DATASET

In [ ]:
dataset = build_dataset('archive/truth_seeker.xlsx', num_class_samples=3000, rnd_state=10)

### PREPROCESS DATA

In [ ]:
dataset = text_edit(dataset,
                    grp_num=True,
                    grp_mention=True,
                    rm_newline=True,
                    rm_punctuation=True,
                    rm_stop_words=True,
                    lowercase=True,
                    lemmatize=False)

### CREATE SAMPLE AND TARGET LISTS

In [ ]:
X = [x['tweet'] for x in dataset.values()]
Y = [x['BinaryNumTarget'] for x in dataset.values()]

### TRAIN/TEST SPLIT

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state = 42)

### GENERATE BAG OF WORDS

In [ ]:
vector_count = CountVectorizer(tokenizer=spacy_tokenizer)
bow_train = vector_count.fit_transform(X_train)
bow_test =  vector_count.transform(X_test)

### HYPERPARAMETER TUNING

In [ ]:
svm_model = tune_svm(bow_train, Y_train)

In [ ]:
lr_model = tune_logistic_regression(bow_train, Y_train)

In [ ]:
mlp_model = MLPClassifier(hidden_layer_sizes=(25, 100, 100, 25), activation='relu', max_iter=5000, random_state=42)

### TRAIN CLASSIFIER

In [ ]:
lr_model.fit(bow_train, Y_train)
lr_y_pred = lr_model.predict(bow_test)
evaluate(Y_test, lr_y_pred)

In [ ]:
svm_model.fit(bow_train, Y_train)
svm_y_pred = svm_model.predict(bow_test)
evaluate(Y_test, svm_y_pred)

In [ ]:
mlp_model.fit(bow_train, Y_train)
mlp_y_pred = mlp_model.predict(bow_test)
evaluate(Y_test, mlp_y_pred)

### TRAIN ENSEMBLE CLASSIFIER

In [ ]:
svm_model = SVC(probability=True, C=100, degree=2, gamma='scale', kernel='poly', random_state=42) 
lr_model = LogisticRegression(C=10, penalty='l2', solver='liblinear', max_iter=5000, random_state=42)
mlp_model = MLPClassifier(hidden_layer_sizes=(25, 100, 100, 25), activation='relu', max_iter=5000, random_state=42)

In [ ]:
svm_model.fit(bow_train, Y_train)
lr_model.fit(bow_train, Y_train)
mlp_model.fit(bow_train, Y_train)

In [ ]:
ensemble_model = VotingClassifier(estimators=[('svm', svm_model), ('logistic', lr_model), ('mlp', mlp_model)], voting='soft')

In [ ]:
ensemble_model.fit(bow_train, Y_train)
ensemble_y_pred = ensemble_model.predict(bow_test)
evaluate(Y_test, ensemble_y_pred)